In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, regexp_replace, trim

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/07/24 20:02:17 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.68 instead (on interface wlp0s20f3)
23/07/24 20:02:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/24 20:02:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

### Criando Dataframes

In [4]:
movies = spark.read.csv(
    "/home/mariannabaldezgomes/Dev/Pessoal/dataset-reviw-movies/dataset/movies.csv",
    header=True,
    sep=",",
    encoding="latin1",
    inferSchema=True
)


In [ ]:
movies.show()

### Renomendo colunas dataframe

In [5]:
rename_movies_columns = {'movieId': 'id', 'title': 'titulo', 'genres':'genero'}

for name, rename in rename_movies_columns.items():
    movies = movies.withColumnRenamed(name, rename)

In [6]:
movies.show()

+---+--------------------+--------------------+
| id|              titulo|              genero|
+---+--------------------+--------------------+
|  1|    Toy Story (1995)|Adventure|Animati...|
|  2|      Jumanji (1995)|Adventure|Childre...|
|  3|Grumpier Old Men ...|      Comedy|Romance|
|  4|Waiting to Exhale...|Comedy|Drama|Romance|
|  5|Father of the Bri...|              Comedy|
|  6|         Heat (1995)|Action|Crime|Thri...|
|  7|      Sabrina (1995)|      Comedy|Romance|
|  8| Tom and Huck (1995)|  Adventure|Children|
|  9| Sudden Death (1995)|              Action|
| 10|    GoldenEye (1995)|Action|Adventure|...|
| 11|American Presiden...|Comedy|Drama|Romance|
| 12|Dracula: Dead and...|       Comedy|Horror|
| 13|        Balto (1995)|Adventure|Animati...|
| 14|        Nixon (1995)|               Drama|
| 15|Cutthroat Island ...|Action|Adventure|...|
| 16|       Casino (1995)|         Crime|Drama|
| 17|Sense and Sensibi...|       Drama|Romance|
| 18|   Four Rooms (1995)|              

### Comandos dataframe viewer

In [ ]:
movies.count() # retorna quantidade de linhas do dataframe

In [ ]:
movies.limit(5).toPandas() # retorna primeiras 5(ou quanto or definido) linhas do dataframe formatadas em pandas

In [ ]:
movies.printSchema() # Retorna type das colunas dos dataframes

### Reformando conteúdo

In [7]:
regex_pattern = r"\((\d{4})\)"

In [8]:
movies = movies.withColumn('ano de lancamento', regexp_extract('titulo', regex_pattern, 1))

In [11]:
movies = movies.withColumn('titulo', regexp_replace('titulo', regex_pattern, '')

SyntaxError: incomplete input (3267870487.py, line 1)

### Selecionando informações

In [ ]:
movies.select('*').show(5, False) # False como parametro no método show mostra a o conteudo completo, ajustando a largura das colunas

In [ ]:
movies.select('titulo', 'genero').show(5) # Visualizando colunas especificas

## a